In [1]:
import pandas as pd
import json
from collections import defaultdict

In [2]:
df_train = pd.read_csv('../../../../../../Medical/PushToNeumann/CSV-Datasets/EBM-Train.csv')
df_test = pd.read_csv('../../../../../../Medical/PushToNeumann/CSV-Datasets/EBM-Test.csv')

In [3]:
df_train

,input_text,target_text
0,Can patients with steatohepatitis take statins...,Another study included patients with biopsy-co...
1,Is the ThinPrep better than conventional Pap s...,The large ALTS trial supports the use of liqui...
2,What is the best approach to a solitary pulmon...,A recent systematic review of studies evaluati...
3,"What evaluation is best for an isolated, enlar...",A study of 550 patients identified 5 significa...
4,What is the best test to diagnose urinary trac...,A study found that urine dipstick testing for ...
...,...,...
1418,Is there much risk in using fluoroquinolones i...,A 1996 study reported the incidence of tendon ...
1419,What is the best treatment for hypertension in...,A review of multiple studies demonstrated that...
1420,What is the role of tacrolimus and pimecrolimu...,A study compared pimecrolimus 1% with betameth...
1421,What is the best regimen for newly diagnosed h...,Results from the largest antihypertensive clin...


In [4]:
import json
instr = '''You are a medical expert. You are given a query and query-relevant information as inputs. Your task is to summarize this information. The summary should be concise, include only non-redundant, query-relevant evidence, and be approximately 100 words long.'''
pre1 = '''\n### Query: '''
pre2 = '''\n### Document: '''
suffix = '''\n### Response:'''

inputs = []
completion = []

for idx in range(df_train.shape[0]):
    try:
        q,sd,rs = df_train.iloc[idx,0].split('\n')[0], ' '.join(df_train.iloc[idx,0].split('\n')[1:]), df_train.iloc[idx,1]
        inputs.append(instr+pre1+q+'?'+pre2+sd+suffix)
        completion.append(rs.replace('\n',' ').strip())
    except:
        print('-------------------')
        print(df_train.loc[idx])


In [5]:
len(inputs),len(completion)

(1423, 1423)

In [6]:
print(inputs[0])

You are a medical expert. You are given a query and query-relevant information as inputs. Your task is to summarize this information. The summary should be concise, include only non-redundant, query-relevant evidence, and be approximately 100 words long.
### Query: Can patients with steatohepatitis take statins?
### Document: To evaluate the efficacy and safety of three hypolipidemic agents in patients with non-alcoholic fatty liver disease associated with hyperlipidemia. Patients with dyslipidemia (Fredrickson type IIb), asymptomatic persistent transaminasemia lasting 24 weeks, and evidence of hepatic fat infiltration on ultrasonography and liver biopsy were studied. Those with predominant hypertriglyceridemia received omega-3 fatty acids (5 mL thrice daily) (Group A), those with predominant hypercholesterolemia received atorvastatin 20 mg/daily (Group B), and overweight patients received orlistat 120 mg thrice daily before meals (Group C). After 24 weeks of treatment, serum transamin

In [7]:
train_inputs = [x for idx,x in enumerate(inputs)]
train_completion = [x for idx,x in enumerate(completion)]

In [8]:
inputs_valid = []
completion_valid = []

df_valid = pd.read_csv('../../../../../../Medical/PushToNeumann/CSV-Datasets/EBM-Valid.csv')
for idx in range(df_valid.shape[0]):
    try:
        q,sd,rs = df_valid.iloc[idx,0].split('\n')[0], ' '.join(df_valid.iloc[idx,0].split('\n')[1:]), df_valid.iloc[idx,1]
        inputs_valid.append(instr+pre1+q+'?'+pre2+sd+suffix)
        completion_valid.append(rs.replace('\n',' ').strip())
    except:
        print('-------------------')
        print(df_valid.loc[idx])



valid_inputs = [x for idx,x in enumerate(inputs_valid)]
valid_completion = [x for idx,x in enumerate(completion_valid)] 

In [9]:
len(train_inputs),len(train_completion),len(valid_inputs),len(valid_completion)

(1423, 1423, 209, 209)

In [10]:
with open('./EBM_Valid.jsonl','w') as f:
    for idx in range(len(valid_inputs)):
        f.write(json.dumps({'prompt':valid_inputs[idx],'completion':valid_completion[idx]})+'\n')
f.close()

In [11]:
PAC_Data = open('../CHQ_/IFT+CHQ_Train_20K.jsonl','r').readlines()
PAC_Data = [json.loads(x) for x in PAC_Data if 'Generate an appropiate title of 15 words or less for the following PubMed Abstract' in json.loads(x)['prompt']]

In [12]:
for data in PAC_Data:
    data['prompt'] = 'You are a medical expert. '+data['prompt']
    data['prompt'] = data['prompt'].replace('### Title:','### Respone:').strip()
    

In [13]:
PAC_Data

[{'prompt': 'You are a medical expert. Generate an appropiate title of 15 words or less for the following PubMed Abstract.\n### PubMed Abstract: Transport of protons and halide ions through planar lipid bilayers made from egg lecithin and a long-chain secondary amine (n-lauryl [trialkylmethyl] amine) in n-decane was studied. Net proton fluxes were measured with a pH electrode, and halide fluxes were measured with 82Br- and 36Cl-. In membranes containing the secondary amine, a large net proton flux was produced either by a Br- gradient with symmetrical pH or by a pH gradient with symmetrical Br-, but not by a pH gradient in Br--free solutions. This H+ flux was electrically silent (nonconductive), and the H+ permeability coefficient was greater than 10(-3) cm sec-1 in 0.1 M NaBr. In Br--free solutions, H+ selectivity was observed electrically by measuring conductances and zero-current potentials generated by H+ activity gradients. The permeability coefficient for this ionic (conductive) 

In [14]:
list_train = []
for inp,target in zip(train_inputs,train_completion):
    list_train.append({'prompt':inp,'completion':target})

In [15]:
list_train[0]

{'prompt': 'You are a medical expert. You are given a query and query-relevant information as inputs. Your task is to summarize this information. The summary should be concise, include only non-redundant, query-relevant evidence, and be approximately 100 words long.\n### Query: Can patients with steatohepatitis take statins?\n### Document: To evaluate the efficacy and safety of three hypolipidemic agents in patients with non-alcoholic fatty liver disease associated with hyperlipidemia. Patients with dyslipidemia (Fredrickson type IIb), asymptomatic persistent transaminasemia lasting 24 weeks, and evidence of hepatic fat infiltration on ultrasonography and liver biopsy were studied. Those with predominant hypertriglyceridemia received omega-3 fatty acids (5 mL thrice daily) (Group A), those with predominant hypercholesterolemia received atorvastatin 20 mg/daily (Group B), and overweight patients received orlistat 120 mg thrice daily before meals (Group C). After 24 weeks of treatment, s

In [17]:
import random
list_dump = list_train #PAC_Data+

random.shuffle(list_dump)

In [18]:
with open('./EBM_Train.jsonl','w') as f:
    for item in list_dump:
        f.write(json.dumps(item)+'\n')